In [30]:
import tensorflow as tf

In [31]:
tf.VERSION

'1.4.0'

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
# INIT WEIGHT

In [6]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [7]:
# INIT BIAS

In [8]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [9]:
# CONV2D

In [10]:
def conv2d(x, W):
    # x --> [batch, H, W, Channels]
    # W --> [kernel H, kernel W, Channel In, Channel Out]
    
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [11]:
# POOLING

In [12]:
def max_pool_2by2(x):
    # x --> [batch, H, W, Channels]
    
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # ksize=[1,2,2,1] la kich thuoc cua max_pool la 2x2, stride la 2

In [13]:
# CONVOLUTIONAL LAYER
def convolutional_layer(input_x, shape): # shape o day la cac chi so cua kernel [H, W, num of channel in, num of kernel]
    W = init_weights(shape)
    b = init_bias([shape[3]]) # number of feature map (same number with kernel)
    return tf.nn.relu(conv2d(input_x, W) + b)

In [14]:
# NORMAL (FULLY CONNECTED)
def normal_full_layer(input_layer, size): # size is number of neuron
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [15]:
# PLACEHOLDER
x = tf.placeholder(tf.float32, shape=[None, 784])

In [16]:
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [17]:
# Layers
x_image = tf.reshape(x, [-1, 28, 28, 1]) # -1 because dont know how many batch of image, 1 is 1 channel

In [18]:
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32]) # Tao ra 32 feature map tu 1 anh, 1 is 1 channel (greyscale) 
convo_1_pooling = max_pool_2by2(convo_1)

In [19]:
convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [20]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 256))

In [21]:
# DROP OUT
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [23]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [24]:
y_pred.shape

TensorShape([Dimension(None), Dimension(10)])

In [25]:
# LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [26]:
# OPTIMIZER
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [27]:
init = tf.global_variables_initializer()

In [ ]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(10)
        
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1)) # argmax(y, 1) là lấy giá trị gần 1 nhất trong list y
            
            # equal trả về True False: [True, False, False] --> [1, 0, 0] để tính accuracy
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))# cast để chuyển qua 0, 1 từ true, false

            # PREDICT [3, 4, 2] TRUE [3, 9, 4]
            # [TRUE, FALSE, FALSE]
            # [1, 0, 0]
            # 0.33
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')